In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib
import numpy as np
import csv

from pathlib import Path
import re
import pandas as pd

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import grela_helpers

In [3]:
conn = grela_helpers.make_connection()

In [4]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [5]:
# establish connection with Google sheets
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ruland_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1od248fJpNbwMO8IVZQVO3xnzrmmTe68yS9XWajyuk5o/edit?usp=sharing")

In [6]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")

In [7]:
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [8]:
lexeme_df = lexeme_df[lexeme_df["target_relemmatized"].str.split().str.len() < 4]

In [9]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2846

In [10]:
# look at a few multiword entries
lexeme_df[lexeme_df["target_len"] > 1].head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
23,Lapis maior,[Lapis maior],lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,"[finde nach etlicher Meynung, die calcinirten ...","[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",2,lapis magnus
24,Leo viridis,[Leo viridis],leo uiridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,"[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",2,leo uiridis
25,Machina tractoria,[Machina tractoria],machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,"[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",2,machina tractoria
30,Panum aereorum species,[Panum aereorum species],panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,"[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...","[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,3,panis aereus species
32,Pyritae aurei coloris,[Pyritae aurei coloris],pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,"[1, è quibus ignis elicitur, è quibus ignis no...","[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,3,pyritus aureus color


In [11]:
# look at entries starting with mercurius
lexeme_df[lexeme_df["target_canonical"].str.startswith("mercurius")]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
27,Mercurius,[Mercurius],mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,"[id est, sulphur., est principium materiale, v...","[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",1,mercurius
96,Mercurius metallorum,[Mercurius metallorum],mercurius metallorum,"[Ruland1612-Mercurius-metallorum, Ruland1612-M...",2,[M],[],[],[{'def': 'Ist darauß die Natur der Coͤrper gez...,2,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[{'entry_id': 'Ruland1612-Mercurius-metallorum...,2,mercurius metallum
1880,Mercurius argentipigmentum,[Mercurius argentipigmentum],mercurius argentipigmentum,[Ruland1612-Mercurius-argentipigmentum],1,[M],[],[],"[{'def': 'ist Schwefel victriol, Alaun Saltz-d...",1,"[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...",[{'entry_id': 'Ruland1612-Mercurius-argentipig...,2,mercurius argentipigmentum
1881,Mercurius chambar,[Mercurius chambar],mercurius chambar,[Ruland1612-Mercurius-chambar],1,[M],[],[],"[{'def': 'ist magnesia, fuessend Coͤrper und W...",1,"[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[{'entry_id': 'Ruland1612-Mercurius-chambar', ...",2,mercurius chaambar
1882,Mercurius corallinus,[Mercurius corallinus],mercurius corallinus,[Ruland1612-Mercurius-corallinus],1,[M],[],[],[{'def': 'qui per oleum ouorum & aquas alias i...,1,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[],2,mercurius corallinus
1883,Mercurius crudus,[Mercurius crudus],mercurius crudus,[Ruland1612-Mercurius-crudus],1,[M],[],[],"[{'def': 'est is, qui nondum separatus est a s...",1,"[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[{'entry_id': 'Ruland1612-Mercurius-crudus', '...",2,mercurius crudus
1884,Mercurius crystallinus,[Mercurius crystallinus],mercurius crystallinus,"[Ruland1612-Mercurius-crystallinus,]",1,[M],[],[],[{'def': 'qui saepe sublimatus est in formam c...,1,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[],2,mercurius crystallinus
1885,Mercurius laxus,[Mercurius laxus],mercurius laxus,[Ruland1612-Mercurius-laxus],1,[M],[],[],"[{'def': 'est turbith minerale.', 'entry_id': ...",1,[est turbith minerale.],[est turbith minerale.],[est turbith minerale.],[],2,mercurius laxus
1886,Mercurius mineralium,[Mercurius mineralium],mercurius mineralium,[Ruland1612-Mercurius-mineralium],1,[M],[],[],"[{'def': 'est oleitas, vel vnctuositas de mine...",1,"[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...",[{'entry_id': 'Ruland1612-Mercurius-mineralium...,2,mercurius mineralis
1887,Mercurius regeneratus,[Mercurius regeneratus],mercurius regeneratus,[Ruland1612-Mercurius-regeneratus],1,[M],[],[],"[{'def': 'est primum ens Mercurii.', 'entry_id...",1,[est primum ens Mercurii.],[est primum ens Mercurii.],[est primum ens Mercurii.],[],2,mercurius regenero


In [12]:
# test with one specific multiword phrase
target_canonical     =  "lapis philosophorum" # "lapis philosophorum"
target_relemmatized  =  "lapis philosophus" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()

(762,)

In [14]:
# 2) phase 2: build detailed KWIC for first N hits
df = grela_helpers.kwic_from_spans(
    conn,
    spans_table="temp_target_spans_table",
    window=10,
    include_tokens=True,
    max_hits=100_000,     # or None, if you know it's safe
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[259006133, 259006134]","[10, 11]","[1, 2]",emlap_100004,emlap_100004_18325,emlap_100004_18322,emlap_100004_18326,. Quaestio An Lapis Philosophicus Ualeat contr...,"[{'token_id': 259006123, 'token_text': '.', 'l...","Utrum lapis philosophorum , si haberetur , ual...","[{'token_id': 259006132, 'token_text': 'Utrum'...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
1,"[lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[257691833, 257691834]","[10, 11]","[3, 4]",emlap_100038,emlap_100038_15321,emlap_100038_15319,emlap_100038_15324,Quod unus est lapis Philosophorum caput Vi Ex ...,"[{'token_id': 257691823, 'token_text': 'Quod',...",Ex quibus extrahatur lapis Philosophorum,"[{'token_id': 257691830, 'token_text': 'Ex', '...",None,Artis auriferae,1572,1572
2,"[lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[255740431, 255740432]","[10, 11]","[11, 12]",emlap_100010,emlap_100010_109,emlap_100010_109,emlap_100010_110,"Ferri spuma , quae a fabris ferrariis abiicitu...","[{'token_id': 255740421, 'token_text': 'Ferri'...","122 Ferri spuma , quae a fabris ferrariis abii...","[{'token_id': 255740420, 'token_text': '122', ...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
3,"[Lapidis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[258207852, 258207853]","[10, 11]","[5, 6]",emlap_100052,emlap_100052_11,emlap_100052_9,emlap_100052_12,e . Mercurius Sapientum : Ubi Magnesiae ( Subi...,"[{'token_id': 258207842, 'token_text': 'e', 'l...",Ubi Magnesiae ( Subiecti uidelicet Lapidis Phi...,"[{'token_id': 258207847, 'token_text': 'Ubi', ...","Khunrath, Heinrich",Confessio de chao physico-chymicorum,1599,1599
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[258109101, 258109102]","[10, 11]","[6, 7]",emlap_100032,emlap_100032_780,emlap_100032_779,emlap_100032_780,suo sulphure occulto : sic etiam illa res de q...,"[{'token_id': 258109091, 'token_text': 'suo', ...",sic etiam illa res de qua lapis philosophorum ...,"[{'token_id': 258109095, 'token_text': 'sic', ...","Ventura, Lorenzo",De ratione conficiendi,1571,1571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[304375077, 304375078]","[10, 11]","[6, 7]",noscemus_631363,noscemus_631363_19500,noscemus_631363_19499,noscemus_631363_19502,"pag . 310 . Pauli Eck , de Sultzbach De Lapide...","[{'token_id': 304375067, 'token_text': 'pag', ...","Pauli Eck , de Sultzbach De Lapide Philosophor...","[{'token_id': 304375071, 'token_text': 'Pauli'...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
758,"[lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[365725081, 365725082]","[10, 11]","[4, 5]",noscemus_630005,noscemus_630005_4015,noscemus_630005_4014,noscemus_630005_4015,"Agricolam Germanice sonat , tractat . autem de...","[{'token_id': 365725071, 'token_text': 'Agrico...",autem de materia & lapide Philosophorum ex Bib...,"[{'token_id': 365725077, 'token_text': 'autem'...","Borel, Pierre",Bibliotheca chimica seu Catalogus librorum phi...,1654,1654
759,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[261799906, 261799907]","[10, 11]","[40, 41]",emlap_100022,emlap_100022_1963,emlap_100022_1963,emlap_100022_1963,", quod est magis naturae de F , ad generandum ...","[{'token_id': 261799896, 'token_text': ',', 'l...","Et cum habueris E. N. O. P , habes materiam no...","[{'token_id': 261799866, 'token_text': 'Et', '...",None,De alchimia opuscula,1550,1550
760,"[lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophu

In [16]:
# test with one specific multiword phrase
target_canonical     =  "aqua" # "lapis philosophorum"
target_relemmatized  =  "aqua" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [16]:
n_spans = conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()[0]
n_spans

310466

In [ ]:
batch_size = 100_000
if n_spans > batch_size:
    all_chunks = []

    for start_idx in range(1, n_spans + 1, batch_size):
        end_idx = min(start_idx + batch_size - 1, n_spans)
        print(f"Processing spans {start_idx}–{end_idx}...")

        df_chunk = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
            span_idx_min=start_idx,
            span_idx_max=end_idx,
        )

        all_chunks.append(df_chunk)

    df = pd.concat(all_chunks, ignore_index=True)
else:
    df = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
        )

Processing spans 1–100000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing spans 100001–200000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
len(df)

In [16]:
def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"

In [17]:
outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [18]:
large_skipped = []  # you can keep this name if you want; it will become "large_batched"

def from_entries_to_file(tc, tr, outdir):
    name = safe_name(tc or tr)
    fpath = str(outdir / f"{name}.parquet")

    grela_helpers.create_target_spans_table(
        conn,
        table_name="temp_target_spans_table",
        target_canonical=tc,
        target_relemmatized=tr,
        emlap_only=False,
        max_hits=1_000_000,  # safeguard
    )

    # kwic_from_spans now batches automatically when out_path is set
    grela_helpers.kwic_from_spans(
        conn,
        spans_table="temp_target_spans_table",
        window=10,
        include_tokens=True,
        out_path=fpath,
        # optional: pass through tuning knobs if you exposed them
        batch_threshold=100_000,
        batch_size=10_000,
    )

In [19]:
tc = "aurum"
tr = "aurum"
from_entries_to_file(tc, tr, outdir)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
tc = "mercurius metallorum"
tr = "mercurius metallum"
from_entries_to_file(tc, tr, outdir)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances

In [22]:
df = read_hits("mercurius_metallorum.parquet")

In [22]:
df = read_hits("aurum.parquet")

69768

In [23]:
pd.DataFrame(df)

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[mercurio, metalla]",relemmatized,lemma,"[mercurius, metallum]","[222037903, 222037904]","[10, 11]","[9, 10]",cc_13979,cc_13979_7508,cc_13979_7507,cc_13979_7508,". Metalla quoque in Mercurium uiuum mutari , e...","[{'token_id': 222037893, 'token_text': '.', 'l...","Metalla quoque in Mercurium uiuum mutari , et ...","[{'token_id': 222037894, 'token_text': 'Metall...","Sennert, Daniel",Epitome naturalis scientiae,1572,1637
1,"[Mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[258817363, 258817364]","[10, 11]","[53, 54]",emlap_100004,emlap_100004_8365,emlap_100004_8365,emlap_100004_8366,"omnibus modis praedominatur ignis , & illud di...","[{'token_id': 258817353, 'token_text': 'omnibu...","Uiuum causat metalla , quanuis unum differt ab...","[{'token_id': 258817310, 'token_text': 'Uiuum'...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
2,"[mercurio, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[260061382, 260061383]","[10, 11]","[53, 54]",emlap_100007,emlap_100007_1504,emlap_100007_1504,emlap_100007_1506,"omnibus moribus predominatur ignis , & illud d...","[{'token_id': 260061372, 'token_text': 'omnibu...",Uiuum causat metalla quamuis adhuc bene differ...,"[{'token_id': 260061329, 'token_text': 'Uiuum'...",None,Rosarium philosophorum,1550,1550
3,"[mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[255746148, 255746149]","[10, 11]","[28, 29]",emlap_100010,emlap_100010_470,emlap_100010_470,emlap_100010_470,"similitudine uerae calcis communis , quia arge...","[{'token_id': 255746138, 'token_text': 'simili...",Compraehendis igitur ex supradictis rationibus...,"[{'token_id': 255746120, 'token_text': 'Compra...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
4,"[mercurium, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[255747996, 255747997]","[10, 11]","[6, 7]",emlap_100010,emlap_100010_569,emlap_100010_568,emlap_100010_569,"sulphure uerba faciemus . Prius diximus , per ...","[{'token_id': 255747986, 'token_text': 'sulphu...","Prius diximus , per calcinationem , mercurium ...","[{'token_id': 255747990, 'token_text': 'Prius'...","Bracesco, Giovanni",De Alchimia Dialogi Duo,1548,1548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,"[Mercurius, Metallorum]",relemmatized,lemma,"[mercurius, metallum]","[341045791, 341045792]","[10, 11]","[14, 15]",noscemus_797708,noscemus_797708_23324,noscemus_797708_23324,noscemus_797708_23326,"uetatus , est gus ex Metallis tractus , Semen ...","[{'token_id': 341045781, 'token_text': 'uetatu...","& praecipitatus Mercurius Metallorum uetatus ,...","[{'token_id': 341045777, 'token_text': '&', 'l...","Sommerhoff, Johann Christoph",Lexicon pharmaceutico-chymicum Latino-Germanic...,1701,1701
62,"[Mercurius, Metallorum]",relemmatized,lemma,"[mercurius, metallum]","[341045808, 341045809]","[10, 11]","[0, 1]",noscemus_797708,noscemus_797708_23329,noscemus_797708_23325,noscemus_797708_23330,num passiuum . Mait . Rul . & Johns . Mercuriu...,"[{'token_id': 341045798, 'token_text': 'num', ...",Mercurius Metallorum .,"[{'token_id': 341045808, 'token_text': 'Mercur...","Sommerhoff, Johann Christoph",Lexicon pharmaceutico-chymicum Latino-Germanic...,1701,1701
63,"[Mercurius, metallorum]",relemmatized,lemma,"[mercurius, metallum]","[317375964, 317375965]","[10, 11]","[4, 5]",noscemus_797709,noscemus_797709_1230,noscemus_797709_1228,noscemus_797709_1230,Schellkraut . Aquesutum Cuscuta Flachsseiden ....,"[{'token_id': 317375954, 'token_text': 'Schell...","Aquila Philosophorum , est Mercurius metalloru...","[{'token_id': 317375960, 'token_text': 'Aquila...","Müller von Löwenstein, Friedrich",Lexicon medico-galeno-chymico-pharmaceuticum c...,1661,1661
64,"[Mercu

In [24]:
lexeme_df["instance_fname"] = lexeme_df["target_canonical"].apply(lambda x: safe_name(x) + ".parquet")

In [25]:
lexeme_df[lexeme_df.duplicated(subset="instance_fname", keep=False)]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname


In [26]:
from pathlib import Path
from datetime import datetime, timedelta

hours = 24
cutoff = datetime.now() - timedelta(hours=hours)

files = []
for p in outdir.rglob("*"):          # use .glob("*") if you only want top-level
    if not p.is_file():
        continue
    mtime = datetime.fromtimestamp(p.stat().st_mtime)
    if mtime >= cutoff:
        files.append((p, mtime, p.stat().st_size))

# newest first
files.sort(key=lambda x: x[1], reverse=True)

print(f"Files modified in the last {hours} hours: {len(files)}")
for p, mtime, size in files[:50]:   # cap print if there are many
    print(f"{mtime:%Y-%m-%d %H:%M:%S}  {size/1e6:8.2f} MB  {p}")

Files modified in the last 24 hours: 543
2025-12-18 20:02:07      0.11 MB  ../data/large_files/emlap_ruland_instances/mercurius_metallorum.parquet
2025-12-18 13:48:25      0.00 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000175001_000200000.parquet
2025-12-18 13:48:24     22.05 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000150001_000175000.parquet
2025-12-18 13:47:43     22.10 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000125001_000150000.parquet
2025-12-18 13:44:48     22.42 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000100001_000125000.parquet
2025-12-18 13:43:39     92.13 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000075001_000100000.parquet
2025-12-18 13:43:00     27.97 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000050001_000075000.parquet
2025-12-18 13:42:15     25.06 MB  ../data/large_files/emlap_ruland_instances/.tmp_an_parts/part_000025001_000

In [27]:
done = [str(f[0]).rpartition("/")[2] for f in files]
done[:5]

['mercurius_metallorum.parquet',
 'part_000175001_000200000.parquet',
 'part_000150001_000175000.parquet',
 'part_000125001_000150000.parquet',
 'part_000100001_000125000.parquet']

In [ ]:
from tqdm.auto import tqdm

pbar = tqdm(total=len(lexeme_df), desc="Lexemes")
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    name = safe_name(tc or tr)
    fname = f"{name}.parquet"

    pbar.set_postfix_str(fname[:60])  # show current file (truncate a bit)

    if fname not in done:
        from_entries_to_file(tc, tr, outdir)
        done.append(fname)

    pbar.update(1)
pbar.close()

Lexemes:   0%|          | 0/2846 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
large_skipped_df = pd.DataFrame(large_skipped)
large_skipped_df

In [ ]:
large_skipped_df.to_csv("../data/large_skipped.csv", index=False)

In [ ]:
len(os.listdir(outdir))

In [ ]:
emlap_metadata = pd.read_csv(
        "https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv",
        sep=";",
    )
emlap_metadata.head(5)

In [ ]:
df = pd.DataFrame(read_hits("lapis_philosophorum.parquet"))
df

In [ ]:
set_with_dataframe(ruland_gs.add_worksheet("casestudy_lapis_philosophorum", 1,1), df)

In [ ]:
df = pd.DataFrame(read_hits("mercurius_metallorum.parquet"))
df

In [ ]:
set_with_dataframe(ruland_gs.add_worksheet("casestudy_mercurius_metallorum", 1,1), df)

In [ ]:
# check instances from noscemus
df[df["grela_id"].str.startswith("noscemus")]

In [ ]:
# find NOSCEMUS entries included in EMLAP
noscemus_emlap_ids = [str(int(x)) for x in emlap_metadata["if_noscemus_id"].unique() if pd.notna(x)]
noscemus_emlap_ids

In [ ]:
noscemus_ruland = ['668514']

In [ ]:
df[df["grela_id"].apply(lambda x: (x[:8] == "noscemus") and  (x[9:] not in noscemus_emlap_ids + noscemus_ruland))]

In [ ]:
csv_path = "../data/emlap_ruland_instances_csv/"
os.makedirs(csv_path, exist_ok=True)

def _is_missing(v) -> bool:
    # safe missing check for scalars; won't choke on list/dict
    return v is None or v is pd.NA or (isinstance(v, float) and np.isnan(v))

def save_to_csv(df_in: pd.DataFrame, fname: str):
    out = df_in.copy()

    json_cols = ["kwic_tokens", "target_sentence_tokens"]
    for c in json_cols:
        if c not in out.columns:
            continue
        out[c] = out[c].map(
            lambda v: ""
            if _is_missing(v)
            else json.dumps(v, ensure_ascii=False, separators=(",", ":"))
            if isinstance(v, (list, dict))
            else str(v)
        )

    # GitHub CSV preview killer: literal newlines inside a cell
    obj_cols = out.select_dtypes(include=["object", "string"]).columns
    for c in obj_cols:
        out[c] = out[c].astype(str).str.replace("\r", "\\r", regex=False).str.replace("\n", "\\n", regex=False)

    out.to_csv(
        os.path.join(csv_path, fname.replace(".parquet", ".csv")),
        sep=",",                 # <- important for GitHub preview
        index=False,
        encoding="utf-8",
        lineterminator="\n",
        quoting=csv.QUOTE_ALL,
        escapechar="\\",
        doublequote=True,
    )

def read_and_filter_hits(fname):
    try:
        df = pd.DataFrame(read_hits(fname))

        grela = df["grela_id"].astype("string")

        nos_mask = grela.str.startswith("noscemus", na=False) & ~grela.str[9:].isin(noscemus_emlap_ids + noscemus_emlap_ids)
        noscemus_instances_N = int(nos_mask.sum())

        cc_instances_N = int(grela.str.startswith("cc", na=False).sum())

        emlap_instances_df = df[grela.str.startswith("emlap", na=False)].copy()

        if not emlap_instances_df.empty:
            save_to_csv(emlap_instances_df, fname)
            emlap_instances = emlap_instances_df.to_dict("records")
            emlap_instances_N = len(emlap_instances)
        else:
            emlap_instances = []
            emlap_instances_N = 0

        return {
            "noscemus_instances_N": noscemus_instances_N,
            "cc_instances_N": cc_instances_N,
            "emlap_instances_N": emlap_instances_N,
            "emlap_instances": emlap_instances,
        }
    except Exception:
        return {"noscemus_instances_N": 0, "cc_instances_N": 0, "emlap_instances_N": 0, "emlap_instances": []}

In [ ]:
%%time
instances_dict_series = pd.DataFrame(lexeme_df["instance_fname"].apply(read_and_filter_hits).tolist())

In [ ]:
lexeme_df = lexeme_df.reset_index(drop=True)
lexeme_merged_df = lexeme_df.join(instances_dict_series)

In [ ]:
lexeme_merged_df[lexeme_merged_df["target_relemmatized"].apply(lambda x: len(x.split()) > 1)]

In [ ]:
lexeme_merged_df.sort_values("emlap_instances_N", ascending=False).head(20)

In [ ]:
lexeme_df = lexeme_merged_df

In [ ]:
lexeme_df["grela_instances_N"] = lexeme_df.apply(lambda x: sum([x["emlap_instances_N"], x["noscemus_instances_N"], x["cc_instances_N"]]), axis=1)


In [ ]:
#emlap_id_title_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['working_title']))
#emlap_id_date_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['date_publication']))

In [ ]:
#def add_emlap_metadata(instances_dict_list):
#    instances_dict_list_new = []
#    for instance_dict in instances_dict_list:
#        id = instance_dict["grela_id"][6:]
#        instance_dict["title"] = emlap_id_title_dict[id]
#        instance_dict["year"] = emlap_id_date_dict[id]
#        instances_dict_list_new.append(instance_dict)
#    return instances_dict_list_new

In [ ]:
# conservative under Google’s 50k hard limit
MAX_CELL = 48000

def _clip(s: str, n: int = MAX_CELL) -> str:
    return s if len(s) <= n else (s[:n-1] + "…")

def _stringify(v) -> str:
    """Controlled stringification so lists/dicts don't explode."""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return ""
    if isinstance(v, (list, dict)):
        # compact JSON, unicode-friendly
        return json.dumps(v, ensure_ascii=False, separators=(',', ':'))
    return str(v)

def prepare_for_sheets(df: pd.DataFrame, *, drop_heavy: bool = False) -> pd.DataFrame:
    """
    Make a sheets-safe copy:
      - optional: drop very heavy/nested cols
      - stringify lists/dicts in a compact way
      - clip any cell > MAX_CELL
    """
    if drop_heavy:
        df = df.drop(columns=[
            # keep this list lean; tweak as you like
            "senses_flat", "Senses", "Translations", "EntryLevelTranslations"
        ], errors="ignore").copy()
    else:
        df = df.copy()

    # stringify then clip per-cell
    df = df.map(_stringify)
    df = df.map(lambda s: _clip(s, MAX_CELL))
    return df

# --- example usage ---
# keep everything but serialize/clip:
df_for_sheet = prepare_for_sheets(lexeme_df.drop(["emlap_instances"], axis=1), drop_heavy=False)

In [ ]:
set_with_dataframe(ruland_gs.add_worksheet("ruland_emlap_instances_2025-12-18", 1,1), df_for_sheet.sort_values("emlap_instances_N", ascending=False))

In [ ]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [ ]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")